# Interactive Map of STEM Colleges

## Install Necessary Packages 

In [ ]:
%pip install geopy folium

## Import Packages

In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
import folium

## Load CSV (colleges) and Convert to Pandas Data Frame

In [151]:
file_name = './stem_college.csv'

college_df = pd.read_csv(file_name)

## Initialize geocoder

In [152]:
geofinder=Nominatim(user_agent="college_mapper")

## Function to get lat and long  

In [153]:
def geoinfo(college):
    location = geofinder.geocode(college)
    if location:
        return location.latitude, location.longitude
    return None, None

## Geocode each college  

In [154]:
college_df[['latitude','longitude']] = college_df.apply(
    lambda row: pd.Series(geoinfo(row['college'])) if pd.isna(row['latitude']) or pd.isna(row['longitude']) else pd.Series([row['latitude'], row['longitude']]),
    axis=1
)

In [155]:
is_file_update = False

if is_file_update:
    college_df[["college",'latitude','longitude','visited_status']].to_csv(file_name,index=False)

## Create map centered at Average location  

In [156]:
map_middle = [college_df['latitude'].mean(), college_df['longitude'].mean()]
map_college = folium.Map(location=map_middle, zoom_start=5)

## Add color to markers

In [157]:
college_df['color'] = college_df['visited_status'].apply(
    lambda x: 'red'
    if x == 'FALSE' else 'blue' 
    if x == 'want' else 'green'
)

## Add College Markers to Map  

In [158]:
for i, row in college_df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup= row['college'],
        icon=folium.Icon(color=row['color'])
    ).add_to(map_college)

In [159]:
map_college